In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from keras.layers.recurrent import LSTM,GRU
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [ ]:
img_size=200
np.random.seed(1234567)
#loading data and fitting follows:
X_train=np.loadtxt('training_data/Xtrain_av_raw200_2_2d0_v2.csv',delimiter=',')
Y_train=np.loadtxt('training_data/Ytrain_av_raw200_2_2d0_v2.csv',delimiter=',')
X_extra=np.loadtxt('training_data/Xextra_av_raw200_2_2d0_v2.csv',delimiter=',')
#Xtrain=Xtrain[:2000]
#X_train=np.transpose(X_train)
print(X_train.shape,Y_train.shape,X_extra.shape)

In [ ]:
kernel_size = 3
kernel = np.ones(kernel_size) / kernel_size
X_train = np.array([np.convolve(X_train[i], kernel, mode='same') for i in range(len(X_train))])
#X_test = [np.convolve(Xtest[i], kernel, mode='same') for i in range(len(Xtest))]
Xtrain, Xtest, Ytrain, Ytest, Xextra_tr, Xextra_tst = train_test_split(X_train, Y_train, X_extra, test_size=0.1)
print(Xtrain.shape,Xtest.shape,Xextra_tr.shape)

In [ ]:
wt=[]
for el in Xtrain:
    med=np.median(el)
    min=el[np.argmin(el)]
    std=np.std(el)
    temp=(el-med)/(med-min)
    clean=[x for x in temp if x > med-std]
    wt.append(1/np.power(np.std(clean),0.25))

norm = np.linalg.norm(np.array(wt))
wt= 1000*(np.array(wt)/norm)



In [ ]:
#test run a RNN  which i dont understand much about, but im hopeful
#Xtrain=np.transpose(Xtrain)
inp = keras.layers.Input(shape=(200,), name="rnn_input")
x=keras.layers.Reshape(target_shape=(200,1))(inp)
x=keras.layers.GRU(20, return_sequences=True,name="GRU_lay_1")(x)
#model.add(LSTM(100, activation='relu', input_shape=(n_in,1)))
#x=keras.layers.RepeatVector(200)(x)
bp=keras.layers.GRU(40, return_sequences=True,name="GRU_lay_2")(x)
#shape_before_flatten = keras.backend.int_shape(bp)[1:]
x=keras.layers.GRU(60,return_sequences=True,name="GRU_lay_3")(x)
bp=keras.layers.TimeDistributed(keras.layers.Dense(1),name='op1')(bp)
#bp=keras.layers.Flatten()(bp)

#op=keras.layers.Dense(32,activation='tanh',name='latentspace')(bp)

#x1=keras.layers.Dense(np.prod(shape_before_flatten),name='dec_1',activation='tanh')(op)
#x1=keras.layers.RepeatVector(200)(op)
#x=keras.layers.Dense(32,activation='relu')(x)
#x=keras.layers.Dense(2,activation='relu',name='op2')(x)
#x1=keras.layers.Reshape(target_shape=shape_before_flatten,name='reshape')(x1)
#x1=keras.layers.LSTM(40, return_sequences=True,name="GRU_layD_2")(x1)
#x1=keras.layers.LSTM(20, return_sequences=True,name="GRU_layD_1")(x1)
#x1=keras.layers.TimeDistributed(keras.layers.Dense(1),name='op2')(x1)

rnn = keras.models.Model(inp, bp, name="rnn")
rnn.summary()

In [ ]:
 rnn.compile(loss='mse', optimizer='adam')

In [ ]:
history=rnn.fit(Xtrain, Xtrain, epochs=20,batch_size=64 , verbose=1,validation_split=0.2,shuffle=True,sample_weight=wt  )

In [ ]:
Ypred=rnn.predict(Xtest)
fig,ax=plt.subplots(10,1,figsize=(10,20))
inc=500
for i in range(10):
    ax[i].plot(Ypred[i+inc,:,0])
    ax[i].plot(Xtest[i+inc,:])

In [ ]:
inp_ex = keras.layers.Input(shape=(3,), name="encoder_input")
x3 = keras.layers.concatenate([op, inp_ex],name='combine_layer')
x3 = keras.layers.Dense(16, activation='tanh',name='class_lay_1')(op)
x3 = keras.layers.Dense(8, activation='relu',name='class_lay_2')(x3)
x3 = keras.layers.Dense(1, activation='relu',name='class_lay_3')(x3)
class_out = keras.layers.Dense(2, activation='softmax',name='class_op_layer')(x3)

full_model=keras.models.Model([inp,inp_ex],class_out,name="classifier")

for layer in full_model.layers[0:6]:
    layer.trainable = False

full_model.summary()

full_model.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.Adam(learning_rate=0.005),metrics='accuracy')

In [ ]:
es_callback = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
#history=vae.fit(Xtrain, Xtrain, epochs=20, batch_size=64 ,verbose=1, validation_split=0.3)
history=full_model.fit([Xtrain,Xextra_tr], Ytrain, epochs=70, batch_size=64 ,verbose=1,shuffle=True, validation_split=0.2)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
#Xtest=np.array(Xtrain).reshape(1138, 7936, 1)
#Xtest=preprocessing.normalize(Xtest)
#Xtest=np.array(Xtest).reshape(358, 2000, 1)
test_loss, test_acc =  full_model.evaluate([np.array(Xtest),np.array(Xextra_tst)],np.array(Ytest))
print('Test accuracy:', test_acc)
Y_pred = full_model.predict([np.array(Xtest),Xextra_tst])
Ypred=np.argmax(Y_pred, axis=1)
Ytest_new=np.argmax(Ytest,axis=1)
cm = confusion_matrix(Ytest_new, Ypred)
print(cm)